<a href="https://colab.research.google.com/github/SojiroNishimura/deeplearning-from-scratch/blob/master/chapter5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 誤差逆伝播法
ニューラルネットワークの学習を行うためには各層での勾配を求める必要がある。勾配は損失関数を数値微分することで求められるが、数値微分の計算は時間がかかる。誤差逆伝播法を使うことで各時点における出力を解析的に微分することができるため、効率的（=高速）に学習が完了する。

## 連鎖律
ある計算をグラフで表現した場合、左(始点)から右(終点)に向けて計算結果を伝達させていくことを**順伝播**という。逆に右(終点)から左(始点)に向けて、入力と各ノードにおける局所的な微分の値を掛け合わせていくことを**逆伝播**という。

$y=f(x)$という計算の場合、逆伝播の入力を$E$とすると$E$とノードの計算を微分したもの=$\frac{\partial y}{\partial x}$を掛け合わせた$E\frac{\partial y}{\partial x}$がそのノードの逆伝播の出力となり、次の(1つ前の）ノードへの入力となる。

### 合成関数
合成関数=複数の関数によって構成される関数。例として$z=(x+y)^2$は以下の2式で構成される。

$z = t^2$

$t = x + y$

連鎖律は以下のように定義される。

> ある関数が合成関数で表される場合、その合成関数の微分は**合成関数を構成するそれぞれの関数の微分の積**によって表すことができる。

上記の式に当てはめると以下のようになる。

$$\frac{\partial z}{\partial x} = \frac{\partial z}{\partial t}\frac{\partial t}{\partial x} = \frac{\partial z}{\partial x}$$

先ほどの式に適用すると以下のとおり。

$$\frac{\partial z}{\partial t} = 2t$$

$$\frac{\partial t}{\partial x} = 1$$

$$\frac{\partial z}{\partial x} = \frac{\partial z}{\partial t}\frac{\partial t}{\partial x} = 2t\cdot1 = 2(x + y)$$

## 逆伝播
「加算」「乗算」の各計算の逆伝播。

### 加算ノードの逆伝播
$z = x + y$の場合

$$\frac{\partial z}{\partial x} = 1$$\
$$\frac{\partial z}{\partial y} = 1$$

右側(上流)からの入力が$\frac{\partial L}{\partial z}$とすると、加算ノードにおける逆伝播の出力はそれぞれ$\frac{\partial L}{\partial z}\cdot 1$となる。

### 乗算ノードの逆伝播
$z = xy$の場合

$$\frac{\partial z}{\partial x} = y$$

$$\frac{\partial z}{\partial y} = x$$

右側(上流)からの入力が$\frac{\partial L}{\partial z}$とすると、逆伝播の出力はそれぞれ以下のようになる。

x方向への逆伝播：$\frac{\partial L}{\partial z}\cdot y$

y方向への逆伝播：$\frac{\partial L}{\partial z}\cdot x$

各ノードは以下のように実装できる。

In [0]:
# 乗算レイヤ
class MulLayer:
  def __init__(self):
    self.x = None
    self.y = None
    
  def forward(self, x, y):
    self.x = x
    self.y = y
    out = x * y
    
    return out
  
  def backward(self, dout):
    dx = dout * self.y # xとyをひっくり返す
    dy = dout * self.x
    
    return dx, dy

In [3]:
# 消費税１０%で1個110円のりんごを2個買う場合の順伝播での計算
apple = 100
apple_num = 2
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [4]:
# 各変数の微分(backward)
dprice = 1

# 順伝播の時とは逆向きに計算を行う
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


In [0]:
# 加算レイヤ
class AddLayer:
  def __init__(self):
    pass
  
  def forward(self, x, y):
    out = x + y
    return out
  
  def backward(self, dout):
    dx = dout * 1
    dy = dout * 1
    return dx, dy

In [6]:
# 消費税10%で100円のりんご2個と150円のみかん3個を買う場合の順伝播での計算
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num) # (1)
orange_price = mul_orange_layer.forward(orange, orange_num) # (2)
all_price = add_apple_orange_layer.forward(apple_price, orange_price) # (3)
price = mul_tax_layer.forward(all_price, tax) # (4)

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice) # (4)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price) # (3)
dorange, dorange_num = mul_orange_layer.backward(dorange_price) # (2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price) # (1)

print(price)
print(dapple_num, dapple, dorange, dorange_num, dtax)

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


## 活性化関数レイヤの実装
活性化関数は各ニューロンへの入力を引数にとり出力に変換する。活性化関数は隠れ層に内包されていると見ることもできるが、ここでは活性化関数も1つの層(レイヤ)とみなす。

### ReLUレイヤ
よく使われる活性化関数として **ReLU(RectifiedL Linear Unit)** がある。これは入力が0以下なら0を出力し、0より大きければ入力値をそのまま出力する。

$
\begin{eqnarray}
y = \left\{
  \begin{array}{ll}
    x ~~~ (x > 0)\\
    0 ~~~ (x \le 0)
  \end{array} \right.
\end{eqnarray}
$

ReLUを微分すると以下のようになる。

$
\begin{eqnarray}
\frac{\partial y}{\partial x} = \left\{
  \begin{array}{ll}
    1 ~~~ (x > 0)\\
    0 ~~~ (x \le 0)
  \end{array} \right.
\end{eqnarray}
$

In [0]:
class Relu:
  def __init__(self):
    self.mask = None
    
  def forward(self, x):
    self.mask = (x <= 0)
    out = x.copy()
    out[self.mask] = 0
    
    return out
  
  def backward(self, dout):
    dout[self.mask] = 0
    dx = dout
    
    return dx

In [10]:
import numpy as np

x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)

mask = (x <= 0)
print(mask)

[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]


### Sigmoidレイヤ
シグモイド関数は以下の式で表される。

$y = \frac{1}{1 + exp(-x)}$

In [0]:
class Sigmoid:
  def __init__(self):
    self.out = None
    
  def forward(self, x):
    out = 1 / (1 + np.exp(-x))
    self.out = out
    
    return out
  
  def backward(self, dout):
    dx = dout * (1.0 - self.out) * self.out
    return dx

## Affine/Softmaxレイヤの実装
### Affineレイヤ
ニューラルネットワークの順伝播で行う行列の積を幾何学の分野で **アフィン変換** と呼ぶ。アフィン変換を行うレイヤ(=順伝播の処理)をAffineレイヤとして定義する。

In [0]:
class Affine:
  def __init__(self, W, b):
    self.W = W # 重み
    self.b = b # バイアス
    self.x = None
    self.dW = None
    self.db = None
    
  def forward(self, x):
    self.x = x
    out = np.dot(x, self.W) + self.b
    
    return out
  
  def backward(self, dout):
    dx = np.dot(dout, self.W.T) # 逆伝播の時は入力に重み行列を転置して掛け合わせる
    self.dW = np.dot(self.x.T, dout)
    self.db = np.sum(dout, axis=0)
    
    return dx

### Softmax-with-Lossレイヤ
ソフトマックス関数は出力値を正規化し、出力値の合計が1になるように出力する。なお学習の際は最終的な出力を行う際にソフトマックス関数を実行しなければならないが、推論を行う際は実行しなくてもよい。これは最終出力直前時点での最大値がソフトマックス関数を実行しても最大であることは自明だからである。なお最終出力直前(正規化しない)の値を **「スコア」** と呼ぶことがある。



In [0]:
def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 

    x = x - np.max(x) # オーバーフロー対策
    return np.exp(x) / np.sum(np.exp(x))
  
def cross_entropy_error(y, t):
  if y.ndim == 1:
      t = t.reshape(1, t.size)
      y = y.reshape(1, y.size)

  # 教師データがone-hot-vectorの場合、正解ラベルのインデックスに変換
  if t.size == y.size:
      t = t.argmax(axis=1)

  batch_size = y.shape[0]
  return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

In [0]:
class SoftmaxWithLoss:
  def __init__(self):
    self.loss = None # 損失
    self.y = None # softmaxの出力
    self.t = None # 教師データ(one-hot vector)
    
  def forward(self, x, t):
    self.t = t
    self.y = softmax(x)
    self.loss = cross_entropy_error(self.y, self.t)
    
    return self.loss
  
  def backward(self, dout=1):
    batch_size = self.t.shape[0]
    dx = (self.y - self.t) / batch_size
    
    return dx

## 誤差逆伝播法の実装
ニューラルネットワークの学習は以下の4ステップで行う。

1. ミニバッチ
  * 訓練データからランダムに一部を選び出す
2. 勾配の算出
  * 各重みパラメータに関する損失関数の勾配を求める
  * 勾配=損失関数を偏微分した結果のベクトル
3. パラメータの更新
  * 重みパラメータを勾配方向に微小量だけ更新する
4. 繰り返す
  * 値が収束するor一定回数1〜3を繰り返す
  
誤差逆伝播法は2の勾配の算出において、数値微分より高速に勾配を計算するために使用する。誤差逆伝播法では、数値微分を行うのではなく解析的に解くことによって高速に計算を行うことができる。



In [0]:
def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 値を元に戻す
        it.iternext()   
        
    return grad

In [0]:
import numpy as np
from collections import OrderedDict

class TwoLayerNet:
  def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
    # 重みの初期化
    self.params = {}
    self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
    self.params['b1'] = np.zeros(hidden_size)
    self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
    self.params['b2'] = np.zeros(output_size)
    
    # レイヤの生成
    self.layers = OrderedDict()
    self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
    self.layers['Relu1'] = Relu()
    self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
    
    self.lastLayer = SoftmaxWithLoss()
    
  def predict(self, x):
    for layer in self.layers.values():
      x = layer.forward(x)
      
    return x
  
  # x:入力データ, t:教師データ
  def loss(self, x, t):
    y = self.predict(x)
    return self.lastLayer.forward(y, t)
  
  def accuracy(self, x, t):
    y = self.predict(x)
    y = np.argmax(y, axis=1)
    if t.ndim != 1 : t = np.argmax(t, axis=1)
      
    accuracy = np.sum(y == t) / float(x.shape[0])
    return accuracy
  
  # x:入力データ, t:教師データ
  def numerical_gradient(self, x, t):
    loss_W = lambda W: self.loss(x, t)
    
    grads = {}
    grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
    grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
    grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
    grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
    
    return grads
  
  def gradient(self, x, t):
    # forward
    self.loss(x, t)
    
    # backward
    dout = 1
    dout = self.lastLayer.backward(dout)
    
    layers = list(self.layers.values())
    layers.reverse()
    for layer in layers:
      dout = layer.backward(dout)
      
    grads = {}
    grads['W1'] = self.layers['Affine1'].dW
    grads['b1'] = self.layers['Affine1'].db
    grads['W2'] = self.layers['Affine2'].dW
    grads['b2'] = self.layers['Affine2'].db
    
    return grads

### 勾配確認
数値微分と誤差逆伝播法での解析的な解法では後者の方が効率的で高速に勾配の計算ができるが、数値微分に比べて実装が複雑になる。数値微分と誤差逆伝播法の計算結果を比較する(両者の差が非常に小さいことを確認する)ことで、誤差逆伝播法の実装に問題がないか確認することができる。これを **勾配確認(gradient check)** と言う。

(実装は割愛)

### 誤差逆伝播法を使った学習


In [17]:
from keras.datasets import mnist
from keras.utils import np_utils

(x_train, t_train), (x_test, t_test) = mnist.load_data()
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

t_train = np_utils.to_categorical(t_train)
t_test = np_utils.to_categorical(t_test)

Using TensorFlow backend.


11493376/11490434 [==============================] - 0s 0us/step


In [18]:
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
  batch_mask = np.random.choice(train_size, batch_size)
  x_batch = x_train[batch_mask]
  t_batch = t_train[batch_mask]
  
  # 誤差逆伝播法によって勾配を求める
  grad = network.gradient(x_batch, t_batch)
  
  # 更新
  for key in ('W1', 'b1', 'W2', 'b2'):
    network.params[key] -= learning_rate * grad[key]
    
  loss = network.loss(x_batch, t_batch)
  train_loss_list.append(loss)
  
  if i % iter_per_epoch == 0:
    train_acc = network.accuracy(x_train, t_train)
    test_acc = network.accuracy(x_test, t_test)
    train_acc_list.append(train_acc)
    test_acc_list.append(test_acc)
    print(train_acc, test_acc)
    
print("End")

0.09751666666666667 0.0974
0.11241666666666666 0.1135
0.11236666666666667 0.1135
0.11236666666666667 0.1135
0.11236666666666667 0.1135
0.11236666666666667 0.1135
0.11236666666666667 0.1135
0.11236666666666667 0.1135
0.11236666666666667 0.1135
0.11236666666666667 0.1135
0.11236666666666667 0.1135
0.11236666666666667 0.1135
0.11236666666666667 0.1135
0.11236666666666667 0.1135
0.11236666666666667 0.1135
0.11236666666666667 0.1135
0.11236666666666667 0.1135
End
